# Visual Transformer with Linformer

Training Visual Transformer on *Dogs vs Cats Data*

* Dogs vs. Cats Redux: Kernels Edition - https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition
* Base Code - https://www.kaggle.com/reukki/pytorch-cnn-tutorial-with-cats-and-dogs/
* Efficient Attention Implementation - https://github.com/lucidrains/vit-pytorch#efficient-attention

In [1]:
!pip -q install vit_pytorch linformer

## Import Libraries

In [2]:
from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

from vit_pytorch.efficient import ViT


In [3]:
print(f"Torch: {torch.__version__}")

Torch: 2.7.0+cu118


In [4]:
import torch
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


Torch: 2.7.0+cu118
CUDA available: True


In [5]:
# Training settings
batch_size = 64
epochs = 100
lr = 3e-5
gamma = 0.7
seed = 42

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

In [7]:
device = 'cuda'

## Load Data

In [17]:
# os.makedirs('data', exist_ok=True)
root_path = r"E:\Project\RemoteSensing\Datasets" 

In [19]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)

# 使用 ImageFolder 加载数据
train_data = datasets.ImageFolder(root=os.path.join(root_path, 'train'), transform=train_transforms)
valid_data   = datasets.ImageFolder(root=os.path.join(root_path, 'val'), transform=val_transforms)
test_data  = datasets.ImageFolder(root=os.path.join(root_path, 'test'), transform=val_transforms)

# DataLoader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader   = DataLoader(valid_data, batch_size=64, shuffle=False)
test_loader  = DataLoader(test_data, batch_size=64, shuffle=False)

In [20]:
print(f"Train: {len(train_data)} images, {len(train_loader)} batches")
print(f"Val:   {len(valid_data)} images, {len(valid_loader)} batches")
print(f"Test:  {len(test_data)} images, {len(test_loader)} batches")
print(f"Classes: {train_data.classes}")

Train: 1259 images, 20 batches
Val:   155 images, 3 batches
Test:  162 images, 3 batches
Classes: ['Beach', 'Bridge', 'Pond', 'Port', 'River']


In [21]:
print(len(train_data), len(train_loader))

1259 20


In [22]:
print(len(valid_data), len(valid_loader))

155 3


## Efficient Attention

### Linformer

In [23]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

### Visual Transformer

In [24]:
model = ViT(
    dim=128,
    image_size=224,
    patch_size=32,
    num_classes=5,
    transformer=efficient_transformer,
    channels=3,
).to(device)

### Training

In [25]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [26]:
checkpoint_dir = 'checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
best_val_acc = 0

In [27]:
def load_checkpoint(checkpoint_path, model, optimizer, scheduler):
    """加载检查点函数"""
    if os.path.exists(checkpoint_path):
        print(f"Loading checkpoint from {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_val_acc = checkpoint['best_val_acc']
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
        return start_epoch, best_val_acc
    else:
        return 0, 0

In [28]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    # 在这里添加检查点保存逻辑
    # 1. 保存最佳模型
    if epoch_val_accuracy > best_val_acc:
        best_val_acc = epoch_val_accuracy
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_val_acc': best_val_acc,
            'loss': epoch_val_loss,
        }
        torch.save(checkpoint, os.path.join(checkpoint_dir, 'best_model.pth'))
        print(f"Saved best model with validation accuracy: {best_val_acc:.4f}")
    
    # 2. 定期保存检查点（每10个epoch）
    if (epoch + 1) % 10 == 0:
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_val_acc': best_val_acc,
            'loss': epoch_val_loss,
        }
        torch.save(checkpoint, os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pth'))
        print(f"Saved regular checkpoint at epoch {epoch+1}")

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )


  0%|          | 0/20 [00:00<?, ?it/s]

Saved best model with validation accuracy: 0.3364
Epoch : 1 - loss : 1.6096 - acc: 0.1902 - val_loss : 1.5582 - val_acc: 0.3364



  0%|          | 0/20 [00:00<?, ?it/s]

Saved best model with validation accuracy: 0.4720
Epoch : 2 - loss : 1.5558 - acc: 0.3026 - val_loss : 1.4855 - val_acc: 0.4720



  0%|          | 0/20 [00:00<?, ?it/s]

...

Epoch : 98 - loss : 0.8664 - acc: 0.6696 - val_loss : 0.7352 - val_acc: 0.7170



  0%|          | 0/20 [00:00<?, ?it/s]

Epoch : 99 - loss : 0.9092 - acc: 0.6509 - val_loss : 0.7041 - val_acc: 0.7195



  0%|          | 0/20 [00:00<?, ?it/s]

Saved regular checkpoint at epoch 100
Epoch : 100 - loss : 0.8610 - acc: 0.6786 - val_loss : 0.6925 - val_acc: 0.7195



In [29]:
# 加载最佳模型并进行测试
best_model_path = os.path.join(checkpoint_dir, 'best_model.pth')
if os.path.exists(best_model_path):
    # 加载最佳模型
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded best model with validation accuracy: {checkpoint['best_val_acc']:.4f}")
    
    # 初始化每个类别的统计
    class_correct = {classname: 0 for classname in train_data.classes}
    class_total = {classname: 0 for classname in train_data.classes}
    
    # 在测试集上评估
    model.eval()
    test_loss = 0
    test_accuracy = 0
    
    with torch.no_grad():
        for data, labels in tqdm(test_loader, desc="Testing"):
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            loss = criterion(outputs, labels)
            
            # 计算总体准确率
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).float()
            test_accuracy += correct.sum() / len(test_loader.dataset)
            test_loss += loss.item() / len(test_loader)
            
            # 计算每个类别的准确率
            for label, prediction, correct_pred in zip(labels, predicted, correct):
                label_class = train_data.classes[label]
                class_correct[label_class] += correct_pred.item()
                class_total[label_class] += 1
    
    # 打印总体结果
    print(f"\nTest Results:")
    print(f"Average Loss: {test_loss:.4f}")
    print(f"Overall Accuracy: {test_accuracy:.4f}")
    
    # 打印每个类别的准确率
    print("\nPer-class Accuracy:")
    for classname in train_data.classes:
        accuracy = class_correct[classname] / class_total[classname]
        print(f"{classname:}: {accuracy:.4f} ({int(class_correct[classname])}/{class_total[classname]})")
else:
    print("No best model checkpoint found!")

Loaded best model with validation accuracy: 0.7494


Testing:   0%|          | 0/3 [00:00<?, ?it/s]


Test Results:
Average Loss: 0.6606
Overall Accuracy: 0.7531

Per-class Accuracy:
Beach: 0.9688 (31/32)
Bridge: 0.5000 (15/30)
Pond: 0.7143 (25/35)
Port: 0.6774 (21/31)
River: 0.8824 (30/34)
